In [1]:
import os
import pandas as pd
import numpy as np
# import sqlite3 as s3
from SQLiteSetUp import (
    SQLMain as sm,
    CleanFuncs as cf)
# from psycopg2 import sql as psql

## Management Tool Selection
* "True" for using PostgreSQL
* "False" for using SQLite

In [2]:
enable_postgres = False

## EXTRACT

In [3]:
# dict of required table changes
def TransformProcess(tab_name, df, enable_postgres):
    tmp = df.copy(deep=True)
    tmp = cf.BasicClean(tmp)
    
    if tab_name == 'ApplicationDocs':
        tmp = tmp.drop('ApplicationDocsURL', axis=1)
        tmp = cf.ConvDateFormat(tmp, ['ApplicationDocsDate'])
        
        if not enable_postgres:
            tmp = tmp.astype({'ApplicationDocsDate': 'str'})

    elif tab_name == 'Products':
        tmp['Method'] = tmp['Form'].apply(lambda x: cf.GetSplitFromCol(x, ';', 1, np.nan))
        tmp['Form'] = tmp['Form'].apply(lambda x: cf.GetSplitFromCol(x, ';', 0, np.nan))
        tmp = cf.RearrangeCol(tmp, 'Method', 'Form')
        
    elif tab_name == 'SubmissionPropertyType':
        tmp = tmp.replace('Null', np.nan)
        
    elif tab_name == 'Submissions':
        tmp = cf.ConvDateFormat(tmp, ['SubmissionStatusDate'])

        if not enable_postgres:
            tmp = tmp.astype({'SubmissionStatusDate': 'str'})
    
    print(f'Data cleansed for {tab_name}.')
    return tmp

In [4]:
raw_dict = dict()
for dirs, _, files in os.walk(os.getcwd()):
    if 'drugs' in dirs:
        for f in files:
            df_name = f.split('.')[0]
            print(f'Importing: {os.path.join(dirs, f)}')
            raw_dict[df_name] = pd.read_csv(os.path.join(dirs, f), delimiter='\t', encoding= 'unicode_escape', error_bad_lines=False)
            
print('\nImport completed.')

Importing: C:\Users\admin\Coding Files\Python Scripts\Script 6\drugsatfda20201117\ActionTypes_Lookup.txt
Importing: C:\Users\admin\Coding Files\Python Scripts\Script 6\drugsatfda20201117\ApplicationDocs.txt
Importing: C:\Users\admin\Coding Files\Python Scripts\Script 6\drugsatfda20201117\Applications.txt
Importing: C:\Users\admin\Coding Files\Python Scripts\Script 6\drugsatfda20201117\ApplicationsDocsType_Lookup.txt
Importing: C:\Users\admin\Coding Files\Python Scripts\Script 6\drugsatfda20201117\MarketingStatus.txt
Importing: C:\Users\admin\Coding Files\Python Scripts\Script 6\drugsatfda20201117\MarketingStatus_Lookup.txt
Importing: C:\Users\admin\Coding Files\Python Scripts\Script 6\drugsatfda20201117\Products.txt
Importing: C:\Users\admin\Coding Files\Python Scripts\Script 6\drugsatfda20201117\SubmissionClass_Lookup.txt

b'Skipping line 35082: expected 8 fields, saw 9\nSkipping line 35083: expected 8 fields, saw 9\nSkipping line 35084: expected 8 fields, saw 9\n'



Importing: C:\Users\admin\Coding Files\Python Scripts\Script 6\drugsatfda20201117\SubmissionPropertyType.txt
Importing: C:\Users\admin\Coding Files\Python Scripts\Script 6\drugsatfda20201117\Submissions.txt
Importing: C:\Users\admin\Coding Files\Python Scripts\Script 6\drugsatfda20201117\TE.txt

Import completed.


## TRANSFORM

In [5]:
clean_dict = dict()
for k,v in raw_dict.items():
    clean_dict[k] = TransformProcess(k, v, enable_postgres)
    
print('\nData tables have been cleansed.')

Data cleansed for ActionTypes_Lookup.
Data cleansed for ApplicationDocs.
Data cleansed for Applications.
Data cleansed for ApplicationsDocsType_Lookup.
Data cleansed for MarketingStatus.
Data cleansed for MarketingStatus_Lookup.
Data cleansed for Products.
Data cleansed for SubmissionClass_Lookup.
Data cleansed for SubmissionPropertyType.
Data cleansed for Submissions.
Data cleansed for TE.

Data tables have been cleansed.


In [6]:
schema_dict = {
    'ApplicationDocs': 'Application',
    'Applications': 'Application',
    'ApplicationsDocsType_Lookup': 'Application',
    'MarketingStatus': 'Marketing',
    'MarketingStatus_Lookup': 'Marketing',
    'TE': 'Marketing',
    'Products': 'Products',
    'Submissions': 'Submissions',
    'SubmissionPropertyType': 'Submissions',
    'SubmissionClass_Lookup': 'Submissions',
    'ActionTypes_Lookup': 'ActionTypes'
}

prim_key_dict = {
    'ApplicationDocs': 'ApplicationDocsID',
    'Applications': 'ApplNo',
    'ApplicationsDocsType_Lookup': 'ApplicationDocsType_Lookup_ID',
    'MarketingStatus': '',
    'MarketingStatus_Lookup': 'MarketingStatusID',
    'TE': '',
    'Products': '',
    'Submissions': '',
    'SubmissionPropertyType': '',
    'SubmissionClass_Lookup': 'SubmissionClassCodeID',
    'ActionTypes_Lookup': 'ActionTypes_LookupID'
}

for_key_dict = {
    'ApplicationDocs': {
        'ApplNo': ('Applications', 'ApplNo'),
        'ApplicationDocsTypeID': ('ApplicationsDocsType_Lookup', 'ApplicationDocsType_Lookup_ID')
    },
    'MarketingStatus': {
        'ApplNo': ('Applications', 'ApplNo'),
        'MarketingStatusID': ('MarketingStatus_Lookup', 'MarketingStatusID')
    },
    'TE': {
        'ApplNo': ('Applications', 'ApplNo'),
        'MarketingStatusID': ('MarketingStatus_Lookup', 'MarketingStatusID')
    },
    'Products': {
        'ApplNo': ('Applications', 'ApplNo'),
        'ProductNo': ('TE', 'ProductNo')
    },
    'Submissions': {
        'ApplNo': ('Applications', 'ApplNo'),
        'SubmissionClassCodeID': ('SubmissionClass_Lookup', 'SubmissionClassCodeID'),
        'SubmissionPropertyType': ('SubmissionPropertyType', 'SubmissionNo')
    },
    'SubmissionPropertyType': {
        'ApplNo': ('Applications', 'ApplNo')
    }
}

---
# <b><font color='red'>POSTGRESQL</font></b>
---

## LOAD

In [7]:
if enable_postgres:
    tab2 = sm.PostgresDB(host= "localhost",
                         database="DrugsFDA",
                         user="postgres",
                         password="root",
                         port="5432")

    tab2.ConnectPGSQL()

In [8]:
%%time

if enable_postgres:
    for tab_name, df in clean_dict.items():
        print(f'Processing upload for {tab_name}...\n')

        tab2.SetSchema(schema_dict.get(tab_name, 'Misc'))     
        tab2.CreateTable(df, tab_name, attr1='prim null', prim_key=prim_key_dict.get(tab_name, ''))
        tab2.BulkInsertRecords(df, tab_name)

        print(f'\nProcessing completed for {tab_name}')
        print('----------------------------------')

Wall time: 0 ns


In [9]:
if enable_postgres:
    tab2.CloseCursor()
    tab2.CloseConnect()

---
# <b><font color='red'>SQLITE</font></b>
---

## LOAD

In [10]:
# for k,v in raw_dict.items():
#     print(f'Dataframe: {k}\n')
#     print(v.head())
#     print('\n----------------\n')

In [11]:
# # convert Date to format 'yyyy/mm/dd' (? and split to individ col of yyyy, mm, dd)
# # prim key = ApplicationDocsID
# a = raw_dict['ApplicationDocs']
# print('# samples: {}\n# unique: \n{}'.format(a.shape[0], a.nunique()))

In [12]:
# # # prim & Foreign key = ApplNo
# a = raw_dict['Applications']
# print('# samples: {}\n# unique: \n{}'.format(a.shape[0], a.nunique()))

In [13]:
# # prim & foreign key = ApplicationDocsType_Lookup_ID
# a = raw_dict['ApplicationsDocsType_Lookup']
# print('# samples: {}\n# unique: \n{}'.format(a.shape[0], a.nunique()))

In [14]:
# # prim key = ApplNo
# # foreign key = MarketingStatusID
# a = raw_dict['MarketingStatus']
# print('# samples: {}\n# unique: \n{}'.format(a.shape[0], a.nunique()))

In [15]:
# # prim & foreign key = MarketingStatusID
# a = raw_dict['MarketingStatus_Lookup']
# print('# samples: {}\n# unique: \n{}'.format(a.shape[0], a.nunique()))

In [16]:
# # for ';' in Form, split and create new columns (form, method)
# # prim key = ApplNo
# a = raw_dict['Products']
# # a['Method'] = a['Form'].apply(lambda x: cf.GetSplitFromCol(x, ';', 1, np.nan))
# # a['Form'] = a['Form'].apply(lambda x: cf.GetSplitFromCol(x, ';', 0, np.nan))
# # a = cf.RearrangeCol(a, 'Method', 'Form')
# # a.head()
# print('# samples: {}\n# unique: \n{}'.format(a.shape[0], a.nunique()))

In [17]:
# # prim key = SubmissionClassCodeID
# a = raw_dict['SubmissionClass_Lookup']
# print('# samples: {}\n# unique: \n{}'.format(a.shape[0], a.nunique()))

In [18]:
# # clean "null" (this is not NaN but string called "Null")
# # prim key = ApplNo
# # foreign key = SubmissionNo
# a = raw_dict['SubmissionPropertyType']
# a.replace('Null', np.nan).head()
# # print('# samples: {}\n# unique: \n{}'.format(a.shape[0], a.nunique()))

In [19]:
# # convert Date to format 'yyyy/mm/dd' (? and split to individ col of yyyy, mm, dd)
# # prim key = ApplNo
# # foreign key = SubmissionClassCodeID, SubmissionNo
# # raw_dict['Submissions']['SubmissionStatusDate'] = raw_dict['Submissions']['SubmissionStatusDate'].apply(lambda x: pd.to_datetime(x, format='%Y/%m/%d'))
# a = raw_dict['Submissions']
# # a.head()
# print('# samples: {}\n# unique: \n{}'.format(a.shape[0], a.nunique()))

In [20]:
# # prim key - ApplNo
# # foreign key = MarketingStatusID
# a = raw_dict['TE'].head()
# print('# samples: {}\n# unique: \n{}'.format(a.shape[0], a.nunique()))

In [21]:
# col_data_dict = set_sql_datatype(raw_dict['Applications'])
# attr_dict = get_sql_col_attr('prim null', raw_dict['Applications'], 'ApplNo')

In [22]:
if not enable_postgres:
    tab1 = sm.SQLiteDB('tab1.db')
    tab1.DBConnect()

Database is set up for: tab1.db


In [23]:
if not enable_postgres:
    for tab_name, df in clean_dict.items():
        print(f'Processing upload for {tab_name}...\n')
        tab1.CreateTable(df, tab_name, attr1='prim null', prim_key=prim_key_dict.get(tab_name, ''), attr2='foreign key', for_key=for_key_dict.get(tab_name, ''))
        tab1.InsertData(df, tab_name)
        print(f'\nProcessing completed for {tab_name}')
        print('----------------------------------')

Processing upload for ActionTypes_Lookup...

CREATE TABLE IF NOT EXISTS ActionTypes_Lookup (
ActionTypes_LookupID INTEGER PRIMARY KEY,
ActionTypes_LookupDescription TEXT NOT NULL,
SupplCategoryLevel1Code TEXT NOT NULL,
SupplCategoryLevel2Code TEXT NOT NULL
);
Table made for: ActionTypes_Lookup
Done data insert into ActionTypes_Lookup

Processing completed for ActionTypes_Lookup
----------------------------------
Processing upload for ApplicationDocs...

ApplNo
('Applications', 'ApplNo')
ApplicationDocsTypeID
('ApplicationsDocsType_Lookup', 'ApplicationDocsType_Lookup_ID')
CREATE TABLE IF NOT EXISTS ApplicationDocs (
ApplicationDocsID INTEGER PRIMARY KEY,
ApplicationDocsTypeID INTEGER NOT NULL,
ApplNo INTEGER NOT NULL,
SubmissionType TEXT NOT NULL,
SubmissionNo INTEGER NOT NULL,
ApplicationDocsTitle TEXT NOT NULL,
ApplicationDocsDate TEXT NOT NULL,
FOREIGN KEY (ApplicationDocsTypeID) REFERENCES ApplicationsDocsType_Lookup (ApplicationDocsType_Lookup_ID)
);
Table made for: ApplicationDoc

In [27]:
# tab1.cursor.execute("SELECT * FROM TE;")
# tables = tab1.cursor.fetchone()
# names = [description[0] for description in tab1.cursor.description]

In [25]:
if not enable_postgres:
    tab1.DBClose()

Database has been closed.


In [27]:
# # set_tab_query = create_table('ApplicationDocs', col_data_dict, attr_dict)
# # set_tab_query = create_table('Applications', col_data_dict, attr_dict)
# # cursor.execute(set_tab_query)

In [28]:
# from contextlib import closing
# with closing(s3.connect('tab2.sqlite3')) as con, con, \
#     closing(con.cursor()) as cur:
#     pass
# # os.remove('tab2.sqlite3')

In [29]:
# from sqlalchemy import create_engine
# from sqlalchemy.pool import NullPool

# to_engine = create_engine('sqlite://', poolclass=NullPool)
# conn = to_engine.connect()
# conn.close()
# to_engine.dispose()

In [30]:
# tab1.InsertData(clean_df, 'ApplicationDocs')

In [31]:
# tab1_df = raw_dict['ApplicationDocs']
# tab2_df = raw_dict['Applications']

In [32]:
# tab1_df.to_sql('ApplicationDocs', conn, if_exists='append', index=False)

In [33]:
# tab1.cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# tables = tab1.cursor.fetchall()
# # tables
# table = pd.read_sql_query("SELECT * from %s WHERE ApplNo='4782'" % tables[0], tab1.conn)

In [34]:
# tab1.cursor.execute("SELECT * FROM ApplicationDocs WHERE ApplNo='4782';")
# q = tab1.cursor.fetchall()